## Web Scraping: The Guardian

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

### Obtain list of news from the coverpage

URL definition:

In [2]:
# url definition
url = "https://www.theguardian.com/uk"

List of news:

In [4]:
# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('h3', class_='fc-item__title')
len(coverpage_news)

101

Now we have a list in which every element is a news article:

In [5]:
coverpage_news[4]

<h3 class="fc-item__title"><a class="fc-item__link" data-link-name="article" href="https://www.theguardian.com/uk-news/2019/dec/28/edinburgh-hogmanay-social-curfew-police"><span class="fc-item__kicker">Hogmanay</span> <span class="u-faux-block-link__cta fc-item__headline"> <span class="js-headline-text">Confusion grows over Edinburgh ‘social curfew’</span></span> </a></h3>

### Let's extract the text from the articles:

First, we'll define the number of articles we want: 

In [6]:
number_of_articles = 5

In [7]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
        
    # We need to ignore "live" pages since they are not articles
    if "live" in coverpage_news[n].find('a')['href']:  
        continue
    
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='content__article-body from-content-api js-article__body')
    x = body[0].find_all('p')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

Let's put them into:

- a dataset which will the input of the models (df_features)
- a dataset with the title and the link (df_show_info)

In [8]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [9]:
df_features

,Article Content
0,Boris Johnson is under renewed pressure to dea...
1,The operator of the Costa del Sol hotel where ...
2,"More than 1,000 celebrities, government employ..."
3,"For over three years, Leo spent his days at th..."
4,Security arrangements for the UK’s biggest new...


In [10]:
df_show_info

,Article Title,Article Link
0,Prison crisis Chaos fuels massive legal costs...,https://www.theguardian.com/society/2019/dec/2...
1,Costa del Sol Hotel insists pool not to blame...,https://www.theguardian.com/uk-news/2019/dec/2...
2,Security blunder Government exposes addresses...,https://www.theguardian.com/uk-news/2019/dec/2...
3,China Inside the jail behind the Christmas ca...,https://www.theguardian.com/world/2019/dec/28/...
4,Hogmanay Confusion grows over Edinburgh ‘soci...,https://www.theguardian.com/uk-news/2019/dec/2...


### Time Elapsed

We are interested in how much time the script takes to get the news because this will impact directly on user experience. For this, we'll put it all into a single function and then call it: 

In [11]:
def get_news_theguardian():
    
    # url definition
    url = "https://www.theguardian.com/uk"
    
    # Request
    r1 = requests.get(url)
    r1.status_code

    # We'll save in coverpage the cover page content
    coverpage = r1.content

    # Soup creation
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    # News identification
    coverpage_news = soup1.find_all('h3', class_='fc-item__title')
    len(coverpage_news)
    
    number_of_articles = 5

    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []

    for n in np.arange(0, number_of_articles):

        # We need to ignore "live" pages since they are not articles
        if "live" in coverpage_news[n].find('a')['href']:  
            continue

        # Getting the link of the article
        link = coverpage_news[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('div', class_='content__article-body from-content-api js-article__body')
        x = body[0].find_all('p')

        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)

    # df_features
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })

    # df_show_info
    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'The Guardian'})

    
    return (df_features, df_show_info)

In [12]:
start = time.time()
x, y = get_news_theguardian()
end =time.time()
te = end-start
print("The time elapsed is %f seconds" %(te))

The time elapsed is 7.467288 seconds
